# Word Prediction

Authors: Ben Shealy

In this notebook we're going to create a basic word prediction model based on __Markov chains__. When it comes to learning algorithms for sequential data like text, the most popular approaches are based on __recurrent neural networks (RNNs)__. However, Markov chains are much simpler so they'll be a great way to learn how to work with sequential data.

This notebook was inspired by the group of freshmen I mentored at HelloWorld 2019. Check out their project [here](https://github.com/JulianChastain/HelloWorld)!

## Markov Chain

A Markov chain is a __directed graph__, or network, where each node is a word and each word has arrows (__edges__) to other words. Each edge has a probability associated with it, and if you look at all of the edges going out of a single word, they should sum to 1. If a word doesn't have a directed edge to another word, that's the same as having an edge with probability of 0.

## Dataset

To construct this Markov chain, we'll need a __corpus__, or dataset of text. We're going to use a dataset consisting of all of Shakespeare's plays, which can be obtained from Kaggle (all you need is `alllines.txt`):

https://www.kaggle.com/kingburrito666/shakespeare-plays

There are plenty of other corpora to choose from -- all of Beyonce's song lyrics, the Bible, Wikipedia, and so on -- and we could even combine multiple corpora. It all depends on what you want to be able to do with your word prediction model. Since we're only going to train our model on Shakespeare, the model's word choices will reflect those of Shakespeare.

Whatever corpus we choose, however, there are some preprocessing steps that must be done. For our particular task, we'll need to split the text into a list of words (also called tokens) so that our model can go through them one at a time. We'll also need to remove punctuation and any other non-word text since we're only concerned with the words themselves.

## Training the Model

To train the Markov chain we need to determine (1) what words should be in the graph and (2) what the probabilities of the edges between each word should be. Remember, we're trying specifically to predict the _next word_ given a _previous word_. So here's what we'll do:

1. Represent each unique word in the corpus as a node in the graph.
2. For each word, the probability of going to another word should be determined by the frequency with which that actually happens in the corpus.

In other words, we just need to look at each _word pair_ that occurs in the text, determine how frequently each word pair occurs, and then use that to compute the probabilities for the outgoing edges of each word. Then, when we want to used the Markov chain to predict the next word from a previous word, we simply look at the previous word in the graph and use the probabilities of it's outgoing edges to select a next word at random. It's like drawing colored marbles from a hat, except there might be more blue marbles than red marbles, etc. The neat thing is that we can provide that next word as the new previous word to get a third word, then a fourth, fifth, and so on. We can use this model to generate endless sequences of text!

There is one more thing to worry about -- what if the previous word isn't in the graph? After all, if we train only on Shakespeare, we probably won't see words like "computer" or "television", so what do we do then? There's no right answer here, but there are a couple of options. We could just return an empty string or throw an error. But that's boring, so how about this: we create a separate probability distribution, called the _default distribution_ or _prior distribution_, which we'll use for words that aren't in the Markov chain. We could construct this distribution a number of different ways, but the simplest thing would probably be to look at the frequencies of individual words in the corpus. Think of this distribution as a catch-all which will get us back into the Markov chain.

All of the code we need for loading and parsing the data, training the model, and prediction next words is implemented in the following code block:

In [ ]:
import numpy as np
import string

def load_dataset(filename):
    f = open(filename, "r", encoding="utf8")
    text = f.read()
    words = text.split()

    words = [word.translate(str.maketrans("", "", string.punctuation)) for word in words]
    words = [word.lower() for word in words]
    
    return words

def make_pairs(words):
    for i in range(len(words) - 1):
        yield (words[i], words[i + 1])

def normalize(prob_dict):
    total = sum(prob_dict.values())

    for word in prob_dict.keys():
        prob_dict[word] /= total
        
def fit(words):
    # create Markov chain
    word_dict = {}

    for word1, word2 in make_pairs(words):
        if word1 not in word_dict:
            word_dict[word1] = {}

        if word2 in word_dict[word1]:
            word_dict[word1][word2] += 1
        else:
            word_dict[word1][word2] = 1

    for prob_dict in word_dict.values():
        normalize(prob_dict)

    # create default distribution
    default_dict = {}

    for word in words:
        if word in default_dict:
            default_dict[word] += 1
        else:
            default_dict[word] = 1

    normalize(default_dict)

    return word_dict, default_dict

def predict(word_dict, default_dict, word):
    word = word.lower()

    # use default distribution if word is not in dictionary
    if word in word_dict:
        prob_dict = word_dict[word]
    else:
        prob_dict = default_dict

    # select next word by sampling from categorical distribution
    next_word = np.random.choice(list(prob_dict.keys()), p=list(prob_dict.values()))

    return next_word

Now we're ready to create our own model! We just have to load the corpus, construct a Markov chain from it, and then we can generate some text.

In [ ]:
# load data
words = load_dataset("alllines.txt")

# create Markov model from data
word_dict, default_dict = fit(words)

In [ ]:
# generate several text sequences from seed words
n_sequences = 10
n_words = 100

for i in range(n_sequences):
    # select a word randomly from the Markov chain
    seed_word = np.random.choice(list(word_dict.keys()))

    # initialize text sequence with seed word
    sequence = [seed_word]

    # repeatedly predict the next word and append it to the sequence
    for i in range(n_words - 1):
        sequence.append(predict(word_dict, default_dict, sequence[-1]))

    print(" ".join(sequence))
    print()

Interesting... I guess. The model can definitely generate endless sequences of text, but they aren't always grammatically correct, and they definitely don't make any sense. But remember that we only used _word pairs_ to train our model. For our model to be more sophisticated, at the very least we would need to look at longer sequences of words. That makes the model more complicated and more computationally expensive to train. But that's what it takes to to natural language processing. And that's why natural language processing is hard.

What can you do to improve our model?